# Find sugar measurements correlations

In [6]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

from select import select

#### Define local functions to use

In [7]:
# TODO

#### Load data from measurements database and prepare it for processing

In [8]:
conn = sqlite3.connect('measurements.sqlite')
tests_bioz = pd.read_sql_query("SELECT timestamp, reference FROM tests where tests.mtype=1", conn)
tests_optical = pd.read_sql_query("SELECT timestamp, reference FROM tests where tests.mtype=0", conn)

test_data_bioz = pd.read_sql_query("SELECT * FROM test_data where test_data.timestamp IN (SELECT timestamp from tests where tests.mtype=1)", conn)
test_data_optical = pd.read_sql_query("SELECT * FROM test_data where test_data.timestamp IN (SELECT timestamp from tests where tests.mtype=0)", conn)
test_data_bioz_pivoted = test_data_bioz.pivot(index='timestamp', columns='parameter', values='value').reset_index()
test_data_bioz_pivoted = test_data_bioz_pivoted.fillna(0) # fill NaNs with zeros
test_data_optical_pivoted = test_data_optical.pivot(index='timestamp', columns='parameter', values='value')
test_data_optical_pivoted = test_data_optical_pivoted.drop([0, 3, 4, 5, 6, 9, 10, 11, 12, 15, 16, 17, 18, 21, 22, 23, 24, 27, 28, 29, 30, 31], axis=1).reset_index()
test_data_optical_pivoted = test_data_optical_pivoted.fillna(0) # fill NaNs with zeros
conn.close()
# prepared data to work with:
merged_bioz = pd.merge(tests_bioz, test_data_bioz_pivoted, on='timestamp', how='left')
merged_optical = pd.merge(tests_optical, test_data_optical_pivoted, on='timestamp', how='left')

#### Play with data here:

In [9]:
#merged_optical.head(10)
# Calculate correlations for merged_bioz (excluding timestamp column)
bioz_correlations = merged_bioz.drop('timestamp', axis=1).corr()['reference'].drop('reference')
print("BioZ Correlations with Reference:")
print(bioz_correlations)
print("\nSum of absolute correlations for BioZ:", abs(bioz_correlations).sum())
print("\n")

BioZ Correlations with Reference:
0    -0.064350
1    -0.045558
2    -0.095118
3    -0.128355
4    -0.113756
5    -0.128013
6    -0.066557
7    -0.086104
8    -0.085602
9    -0.085478
10   -0.060701
Name: reference, dtype: float64

Sum of absolute correlations for BioZ: 0.9595927395117537




In [10]:
optical_correlations = merged_optical.drop('timestamp', axis=1).corr()['reference'].drop('reference')
print("Optical Correlations with Reference:")
print(optical_correlations)
print("\nSum of absolute correlations for Optical:", abs(optical_correlations).sum())

Optical Correlations with Reference:
1    -0.029620
2    -0.051530
7    -0.004866
8    -0.050211
13   -0.041978
14   -0.028698
19   -0.030229
20   -0.020903
25   -0.024944
26   -0.021393
Name: reference, dtype: float64

Sum of absolute correlations for Optical: 0.304372749135523


#### Train/test/predict